In [1]:
#pip install selenium

In [2]:
import json
import pandas as pd
import gzip
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from bs4 import BeautifulSoup

from requests import get
import requests

from selenium import webdriver
import time

from random import randint
from time import sleep

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [4]:
df.head(60)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011"
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013"
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013"
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014"
5,A27UF1MSF3DB2,0000031887,"C-Lo ""Cynthia""","[0, 0]",I received this today and I'm not a fan of it ...,4.0,It's ok,1396224000,"03 31, 2014"
6,A16GFPNVF4Y816,0000031887,design maven,"[0, 0]",Bought this as a backup to the regular ballet ...,5.0,Great for dress-up and for ballet practice,1399075200,"05 3, 2014"
7,A2M2APVYIB2U6K,0000031887,Jamie P.,"[0, 0]",Great tutu for a great price. It isn't a &#34;...,5.0,Great value,1356220800,"12 23, 2012"
8,A1NJ71X3YPQNQ9,0000031887,JBerger,"[0, 0]","My daughter liked this, and it with her costum...",4.0,Good,1384041600,"11 10, 2013"
9,A3EERSWHAI6SO,0000031887,"Jeffrey Hollingshead ""Jillian hollingshead""","[7, 8]",For what I paid for two tutus is unbeatable an...,5.0,WOW !! ..is all I have to say!,1349568000,"10 7, 2012"


In [5]:
#Check size of dataset 
df.shape

(278677, 9)

In [5]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [6]:
df[df['asin'] == 'B00971NLBS']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
224886,A1J4GHI1VRLCX9,B00971NLBS,Alya,"[1, 1]",I will be purchasing more of this brand of shi...,5.0,Classic,1374624000,"07 24, 2013"
224887,A21UXFFQGQS4YJ,B00971NLBS,"Anne Harris ""Lady H""","[0, 0]",I found it to be a little roomy but it will be...,5.0,nice dress shirt,1393545600,"02 28, 2014"
224888,A22Z5LSEIPN8DU,B00971NLBS,Ashleyy,"[0, 0]","I ordered this shirt in a light blue, size two...",2.0,"Wrong color, wrong size.",1372291200,"06 27, 2013"
224889,A3SP43WB89B2UE,B00971NLBS,"cecilafayette ""ceci""","[0, 0]","This shirt fits well, looks nice, and is a gre...",4.0,Great Little Basic Shirt,1388102400,"12 27, 2013"
224890,A1Q3MDPIS017JE,B00971NLBS,Kay Z,"[0, 0]","The material is a little thin, but that was ex...",4.0,"Good fit, little thin",1400284800,"05 17, 2014"
224891,A1BCJXOINBPB38,B00971NLBS,"lascrucesgirl ""Growing boy's momma""","[3, 4]",I did not use the tie for my son. It was dress...,5.0,Nicely made and looks sharp.,1360627200,"02 12, 2013"
224892,A1AFORE8L2LGGW,B00971NLBS,mandamommyof3,"[5, 5]",This is adorable! I would buy more if the tie...,4.0,Tie is too short,1356652800,"12 28, 2012"
224893,A22E4B018G6NX7,B00971NLBS,RDeering,"[0, 0]",Nice little shirt I couldn't find elsewhere th...,5.0,can't complain,1388966400,"01 6, 2014"


In [7]:
#Check how many unique items are in the data 
len(df['asin'].unique())

23033

In [6]:
#Created a non-repeating list of product ID #s with the intent of looping through them in the url to scrape. 
asin_set = set(df['asin'])
asin_set = list(asin_set)

In [9]:
asin_set

['0000031887',
 '0123456479',
 '1608299953',
 '1617160377',
 'B00001W0KA',
 'B00001WRHJ',
 'B00004SR8W',
 'B00004SR8Z',
 'B00004SR9P',
 'B00004U1J2',
 'B000051SEN',
 'B000051SEP',
 'B00005JHKE',
 'B00005JSBK',
 'B00005KJXN',
 'B00005TQI7',
 'B0000643Q8',
 'B000067R84',
 'B00006I551',
 'B00006XXGO',
 'B0000722HK',
 'B000072UMJ',
 'B000074RL3',
 'B000078CYM',
 'B00007AS82',
 'B00007FFL9',
 'B00007GD8X',
 'B00007GD9I',
 'B00007GD9W',
 'B00007GDAL',
 'B00007GDD3',
 'B00007GDG5',
 'B00007IVVR',
 'B000086211',
 'B0000862FI',
 'B0000864CZ',
 'B0000865II',
 'B0000866JI',
 'B000086778',
 'B0000867AN',
 'B0000867GG',
 'B0000867ON',
 'B0000868O9',
 'B000086910',
 'B00008695M',
 'B0000869QI',
 'B000086Q9O',
 'B0000891IO',
 'B0000891K0',
 'B00008AALU',
 'B00008ECKG',
 'B00008I8YM',
 'B00008ID0B',
 'B00008ID0L',
 'B00008ID1O',
 'B00008ID39',
 'B00008IEUW',
 'B00008IOHR',
 'B00008IPCG',
 'B00008KH9C',
 'B00008KI3U',
 'B00008LUS5',
 'B00009ESZI',
 'B00009OXE8',
 'B00009QB5V',
 'B00009QMR3',
 'B00009R8

In [15]:
#Check how many unique users are in the data. 
len(df['reviewerID'].unique())

39387

EDA

In [16]:
#Check columns types 
df.dtypes

reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
dtype: object

In [17]:
#Convert universal time to date
df['clean_time'] = pd.to_datetime(df['unixReviewTime'], unit='s')
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,clean_time
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011",2011-02-12
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013",2013-01-19
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013",2013-01-04
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014",2014-04-27
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014",2014-03-15


In [18]:
#Drop unuseful date and time columns 
df.drop(columns=['reviewTime', 'unixReviewTime'], inplace=True)

In [19]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'clean_time'],
      dtype='object')

In [20]:
df['reviewText'].head()

0    This is a great tutu and at a really great pri...
1    I bought this for my 4 yr old daughter for dan...
2    What can I say... my daughters have it in oran...
3    We bought several tutus at once, and they are ...
4    Thank you Halo Heaven great product for Little...
Name: reviewText, dtype: object

Scrape additional data using Selenium

In [3]:
driver = webdriver.Chrome(executable_path="../chromedriver/macos/chromedriver") 

In [18]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [24]:
t1 = time.time()
print(t1)
list_of_dicts = []
for i in redo_list[8528:10000]:
    print(f'https://www.amazon.com/gp/product/{i}')
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    product_info = {}
    try:
        product_info['asin'] = i
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()

        product_info['description'] = soup.find('ul', {'class':'a-unordered-list a-vertical a-spacing-none'}).text.strip()
        
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).text.strip()
        
        product_info['size'] = soup.find('div', {'id': 'variation_size_name'})
        
        product_info['details'] = soup.find_all('td', {'class': 'a-span7 a-size-base'})
        
        if soup.find('div', {'id': 'variation_color_name'}):
            product_info['color'] = soup.find('div', {'id': 'variation_color_name'})
            list_of_dicts.append(product_info)
        
        elif soup.find_all('span', {'class': 'selection'}):
            product_info['color'] = soup.find_all('span', {'class': 'selection'})
            list_of_dicts.append(product_info)
            
        elif soup.find_all('div', {'class': 'a-row a-spacing-micro'}):
            product_info['color'] = soup.find_all('div', {'class': 'a-row a-spacing-micro'})
            list_of_dicts.append(product_info)
        else:
            product_info['color'] = 'unavailable'
            list_of_dicts.append(product_info)
    
    except:
        product_info['asin'] = i
        
        product_info['name'] = 'unknown product name'

        product_info['description'] = 'unknown details'
        
        product_info['category'] = 'unknown category'
        
        product_info['color'] = 'unknown color' 
        list_of_dicts.append(product_info)
        print(len(list_of_dicts))
    sleep(randint(1,2))
    

print(time.time() - t1)

1565647306.5202
https://www.amazon.com/gp/product/B009L4OGDS
https://www.amazon.com/gp/product/B009L5081Q
https://www.amazon.com/gp/product/B009L53QXI
https://www.amazon.com/gp/product/B009L5YU7O
https://www.amazon.com/gp/product/B009LB8CX6
https://www.amazon.com/gp/product/B009LDP8UO
https://www.amazon.com/gp/product/B009LDP93A
https://www.amazon.com/gp/product/B009LDP9JY
https://www.amazon.com/gp/product/B009LEFPAQ
9
https://www.amazon.com/gp/product/B009LEFPBA
https://www.amazon.com/gp/product/B009LEFR4A
11
https://www.amazon.com/gp/product/B009LEFS26
12
https://www.amazon.com/gp/product/B009LEG2EO
13
https://www.amazon.com/gp/product/B009LEG3XO
14
https://www.amazon.com/gp/product/B009LJAJAW
https://www.amazon.com/gp/product/B009LL8NQC
https://www.amazon.com/gp/product/B009LM7BWS
17
https://www.amazon.com/gp/product/B009LNATJY
18
https://www.amazon.com/gp/product/B009LT2DEC
https://www.amazon.com/gp/product/B009LT4S9U
https://www.amazon.com/gp/product/B009LTL5VE
https://www.amazon.

177
https://www.amazon.com/gp/product/B009S3HYQ8
178
https://www.amazon.com/gp/product/B009S3I7EG
179
https://www.amazon.com/gp/product/B009S4D196
https://www.amazon.com/gp/product/B009S4GJ6I
https://www.amazon.com/gp/product/B009S4GPR6
https://www.amazon.com/gp/product/B009S933NA
https://www.amazon.com/gp/product/B009SFVKOS
184
https://www.amazon.com/gp/product/B009SIPDIE
https://www.amazon.com/gp/product/B009SJBZM6
https://www.amazon.com/gp/product/B009SL361C
https://www.amazon.com/gp/product/B009SLGGCI
https://www.amazon.com/gp/product/B009SN0HBM
https://www.amazon.com/gp/product/B009SNXFJ8
190
https://www.amazon.com/gp/product/B009SPN2TO
https://www.amazon.com/gp/product/B009SR3J68
192
https://www.amazon.com/gp/product/B009SRN6T8
https://www.amazon.com/gp/product/B009SRQ7YO
https://www.amazon.com/gp/product/B009SSD1FQ
195
https://www.amazon.com/gp/product/B009SSDTMQ
https://www.amazon.com/gp/product/B009STP6YO
https://www.amazon.com/gp/product/B009SZLCHS
198
https://www.amazon.com/

https://www.amazon.com/gp/product/B00A044PK2
https://www.amazon.com/gp/product/B00A09C3BK
355
https://www.amazon.com/gp/product/B00A0D4V7K
https://www.amazon.com/gp/product/B00A0D5LXS
https://www.amazon.com/gp/product/B00A0DFREQ
https://www.amazon.com/gp/product/B00A0GLAM6
https://www.amazon.com/gp/product/B00A0GTAVO
https://www.amazon.com/gp/product/B00A0I8G08
https://www.amazon.com/gp/product/B00A0I9XE6
362
https://www.amazon.com/gp/product/B00A0L8TL6
363
https://www.amazon.com/gp/product/B00A0M17XM
364
https://www.amazon.com/gp/product/B00A0NQ1EQ
365
https://www.amazon.com/gp/product/B00A0NQGN2
366
https://www.amazon.com/gp/product/B00A0O54DY
367
https://www.amazon.com/gp/product/B00A0O5IPS
368
https://www.amazon.com/gp/product/B00A0OJ71Y
https://www.amazon.com/gp/product/B00A0SXLOO
https://www.amazon.com/gp/product/B00A0U15N6
371
https://www.amazon.com/gp/product/B00A0V0LGM
372
https://www.amazon.com/gp/product/B00A0ZN1LA
https://www.amazon.com/gp/product/B00A10HLCE
https://www.ama

https://www.amazon.com/gp/product/B00A8FCVEK
https://www.amazon.com/gp/product/B00A8GAATC
https://www.amazon.com/gp/product/B00A8JOAVS
https://www.amazon.com/gp/product/B00A8KSWCA
https://www.amazon.com/gp/product/B00A8LCJHI
535
https://www.amazon.com/gp/product/B00A8LSH0G
https://www.amazon.com/gp/product/B00A8ODE7Y
537
https://www.amazon.com/gp/product/B00A8VZM40
538
https://www.amazon.com/gp/product/B00A8WJKR4
539
https://www.amazon.com/gp/product/B00A8YXHKS
540
https://www.amazon.com/gp/product/B00A8YXLPE
541
https://www.amazon.com/gp/product/B00A8YYEAK
542
https://www.amazon.com/gp/product/B00A8YZFT4
543
https://www.amazon.com/gp/product/B00A8ZDWG6
544
https://www.amazon.com/gp/product/B00A97M8W2
https://www.amazon.com/gp/product/B00A97X6Y6
https://www.amazon.com/gp/product/B00A9LO2F4
547
https://www.amazon.com/gp/product/B00A9N3QB8
548
https://www.amazon.com/gp/product/B00A9OXDVK
https://www.amazon.com/gp/product/B00A9Q4TJS
550
https://www.amazon.com/gp/product/B00A9QSXW2
551
htt

702
https://www.amazon.com/gp/product/B00AG0OS1G
703
https://www.amazon.com/gp/product/B00AG0QQZC
704
https://www.amazon.com/gp/product/B00AG0RZ2K
705
https://www.amazon.com/gp/product/B00AG0TA1O
706
https://www.amazon.com/gp/product/B00AG2EEZO
707
https://www.amazon.com/gp/product/B00AG2HBG8
708
https://www.amazon.com/gp/product/B00AG327V6
709
https://www.amazon.com/gp/product/B00AG34XJ0
710
https://www.amazon.com/gp/product/B00AG3ZI4E
711
https://www.amazon.com/gp/product/B00AG415QI
712
https://www.amazon.com/gp/product/B00AG41XWO
713
https://www.amazon.com/gp/product/B00AG42DN2
714
https://www.amazon.com/gp/product/B00AG4BTFA
715
https://www.amazon.com/gp/product/B00AGS1E6U
716
https://www.amazon.com/gp/product/B00AGY1M34
717
https://www.amazon.com/gp/product/B00AH02J3Y
718
https://www.amazon.com/gp/product/B00AH22YSM
719
https://www.amazon.com/gp/product/B00AH3DCAA
720
https://www.amazon.com/gp/product/B00AH4QHOM
721
https://www.amazon.com/gp/product/B00AH9W0FM
722
https://www.amaz

https://www.amazon.com/gp/product/B00ANBOOTE
872
https://www.amazon.com/gp/product/B00ANBU0GK
873
https://www.amazon.com/gp/product/B00ANBWGEO
874
https://www.amazon.com/gp/product/B00AND8AFG
875
https://www.amazon.com/gp/product/B00AND8W9A
876
https://www.amazon.com/gp/product/B00ANHGGB2
https://www.amazon.com/gp/product/B00ANI0LZI
878
https://www.amazon.com/gp/product/B00ANJ3ZGO
879
https://www.amazon.com/gp/product/B00ANJJB0S
880
https://www.amazon.com/gp/product/B00ANKMMNU
https://www.amazon.com/gp/product/B00ANKOVM0
882
https://www.amazon.com/gp/product/B00ANKOVN4
883
https://www.amazon.com/gp/product/B00ANKW4RE
884
https://www.amazon.com/gp/product/B00ANL5BKU
885
https://www.amazon.com/gp/product/B00ANMBA40
https://www.amazon.com/gp/product/B00ANNGWRY
887
https://www.amazon.com/gp/product/B00ANQKZM4
888
https://www.amazon.com/gp/product/B00ANTJRG6
889
https://www.amazon.com/gp/product/B00ANTSXTI
890
https://www.amazon.com/gp/product/B00ANTW5CE
891
https://www.amazon.com/gp/produc

1040
https://www.amazon.com/gp/product/B00ASQ1P6E
1041
https://www.amazon.com/gp/product/B00ASQZN6M
1042
https://www.amazon.com/gp/product/B00AT6HGX4
1043
https://www.amazon.com/gp/product/B00AT90IOK
1044
https://www.amazon.com/gp/product/B00ATB16NK
1045
https://www.amazon.com/gp/product/B00ATCB1KM
1046
https://www.amazon.com/gp/product/B00ATCBS1Y
1047
https://www.amazon.com/gp/product/B00ATCBUFI
1048
https://www.amazon.com/gp/product/B00ATCBUGC
1049
https://www.amazon.com/gp/product/B00ATD8NT8
1050
https://www.amazon.com/gp/product/B00ATEVG2I
1051
https://www.amazon.com/gp/product/B00ATFC9UA
https://www.amazon.com/gp/product/B00ATFOZ70
1053
https://www.amazon.com/gp/product/B00ATG2SAK
1054
https://www.amazon.com/gp/product/B00ATHEMTY
https://www.amazon.com/gp/product/B00ATJU0V6
https://www.amazon.com/gp/product/B00ATK5Z7Y
1057
https://www.amazon.com/gp/product/B00ATL5Z0U
1058
https://www.amazon.com/gp/product/B00ATNIVFO
1059
https://www.amazon.com/gp/product/B00ATPE8CW
1060
https://ww

1207
https://www.amazon.com/gp/product/B00AZFCYUO
1208
https://www.amazon.com/gp/product/B00AZGGPXA
1209
https://www.amazon.com/gp/product/B00AZHN460
https://www.amazon.com/gp/product/B00AZHRJS4
1211
https://www.amazon.com/gp/product/B00AZJZVX2
1212
https://www.amazon.com/gp/product/B00AZK2RO2
1213
https://www.amazon.com/gp/product/B00AZKOG8C
1214
https://www.amazon.com/gp/product/B00AZKUC7G
1215
https://www.amazon.com/gp/product/B00AZN4K80
1216
https://www.amazon.com/gp/product/B00AZOC7QQ
1217
https://www.amazon.com/gp/product/B00AZOFV00
1218
https://www.amazon.com/gp/product/B00AZOQUIM
1219
https://www.amazon.com/gp/product/B00AZORS9W
https://www.amazon.com/gp/product/B00AZOZHWW
1221
https://www.amazon.com/gp/product/B00AZPOIF8
https://www.amazon.com/gp/product/B00AZPPEQ0
1223
https://www.amazon.com/gp/product/B00AZPZQ3G
1224
https://www.amazon.com/gp/product/B00AZU1GWG
1225
https://www.amazon.com/gp/product/B00AZWFDT6
1226
https://www.amazon.com/gp/product/B00B00HU8O
1227
https://ww

1374
https://www.amazon.com/gp/product/B00B5HUIO0
https://www.amazon.com/gp/product/B00B5I0M3G
1376
https://www.amazon.com/gp/product/B00B5MHIRA
1377
https://www.amazon.com/gp/product/B00B5MHQCW
1378
https://www.amazon.com/gp/product/B00B5MJ4CM
1379
https://www.amazon.com/gp/product/B00B5MJBPC
1380
https://www.amazon.com/gp/product/B00B5MJSJQ
1381
https://www.amazon.com/gp/product/B00B5MJWF6
1382
https://www.amazon.com/gp/product/B00B5MK7N2
1383
https://www.amazon.com/gp/product/B00B5W7FD2
1384
https://www.amazon.com/gp/product/B00B5W817Q
1385
https://www.amazon.com/gp/product/B00B5WF3TU
1386
https://www.amazon.com/gp/product/B00B5WZ89U
1387
https://www.amazon.com/gp/product/B00B5XRD9M
1388
https://www.amazon.com/gp/product/B00B5YU438
1389
https://www.amazon.com/gp/product/B00B5ZT6I6
1390
https://www.amazon.com/gp/product/B00B60MQL4
1391
https://www.amazon.com/gp/product/B00B61KGNS
1392
https://www.amazon.com/gp/product/B00B61VSYE
1393
https://www.amazon.com/gp/product/B00B63PXOS
1394


In [25]:
#Check the length of the scrape 
len(list_of_dicts)

1472

In [26]:
#Check the know number of "unknown" rows to make sure it's low 
pd.DataFrame(list_of_dicts)['category'].value_counts()

unknown category                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              976
Clothing, Shoes & Jewelry\n            \n\n\n             ›\n         \n\n\n                Women\n            \n\n\n             ›\n         \n\n\n                Shoes\n            \n\n\n             ›\n         \n\n\n                 Pumps                                                                                                                                                                                                                                                             19
Clot

In [14]:
# Save first 8000 scrapes to a DataFrame and then to a csv
# product8000 = pd.DataFrame(list_of_dicts)

# product8000.to_csv('product8000.csv')

In [22]:
# Saved next 534 scrapes to a DataFrame and then to a csv
# product8534 = pd.DataFrame(list_of_dicts)

# product8534.to_csv('data/product8534')

In [29]:
# Saved next 6000 scrapes to a DataFrame and then to a csv
# product14000 = pd.DataFrame(list_of_dicts)

# product14000.to_csv('product14000.csv')

In [35]:
# Saved another 6000 scrapes to a DataFrame and then to a csv
# product20000 = pd.DataFrame(list_of_dicts)

# product20000.to_csv('product20000.csv')

In [41]:
# Saved next 1180 scrapes to a DataFrame and then to a csv
# product21180 = pd.DataFrame(list_of_dicts)

# product21180.to_csv('product21180.csv')

In [46]:
# Saved last section of scrapes to a DataFrame and then to a csv

# product23033 = pd.DataFrame(list_of_dicts)

# product23033.to_csv('product23033.csv')

Many of the above scrapes were blocked by an anti-robot page so i'll need to go through and check for unavailable rows and make a new list of asins to rescrape.

In [4]:
# Function to remove html
def remove_html(text):
    try:
        soup = BeautifulSoup(text, 'lxml')
        html_free = soup.get_text(strip=True)
        return html_free
    except:
        return "missing"

In [5]:
# Function to clean the DataFrame
def EDA(df):
    # Remove html and strip off brackets and 'Color:' from color column
    df['color'] = df['color'].apply(lambda x: remove_html(x)).str.replace(']', '').str.replace('[', '').str.replace('Color:', '')
    
    # Replace escape characters and white space in Category column 
    df['category'] = df['category'].str.replace('\n', '').str.replace('  ', '')
    
    # Split Categroy column on the '›' symbol, up to 6 times and return them in a new df where each split is a new column 
    category = df['category'].str.split("›", n=6, expand=True)
    
    # Rename each category split column and add it onto the original df
    df['department'] = category[0]
    df['demographic'] = category[1]
    df['division'] = category[2]
    df['category'] = category[3]
    df['subcategory'] = category[4]
    df['type'] = category[5]
    df['detail_type'] = category[6]
    
    # Remove special characters from the description column 
    df['description'] = df['description'].str.replace('\t', '').str.replace('\n', '')
    
    # Remove html and strip off brackets from details column
    df['details'] = df['details'].apply(lambda x: remove_html(x)).str.replace(']', '').str.replace('[', '')
   
    #Remove html and cut off 'Size:' from the size column 
    df['size'] = df['size'].apply(lambda x: remove_html(x)).str.replace('Size:', '')
    
    return df.head()

In [6]:
product8000 = pd.read_csv('data/product8000.csv')

In [7]:
product20000 = pd.read_csv('data/product20000.csv')

In [8]:
product21180 = pd.read_csv('data/product21180.csv')

In [9]:
product23033 = pd.read_csv('data/product23033.csv')

In [10]:
#A list of DataFrames to be concatonated
dataframes = [product8000, product20000, product21180, product23033]

# Concat all 4 DataFrames into one, along the row axis.
total_product = pd.concat(dataframes, axis=0)

In [11]:
total_product.shape

(17033, 8)

In [12]:
EDA(total_product)

,Unnamed: 0,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,0,0000031887,Dance,unavailable,This fits your . Make sure this fitsby ent...,,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,missing,Sports & Outdoors,Sports & Fitness,Other Sports,Clothing,Girls,Skirts
1,1,0123456479,Jewelry Boxes & Organizers,unavailable,This fits your . Make sure this fitsby ent...,,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,missing,"Clothing, Shoes & Jewelry","Shoe, Jewelry & Watch Accessories",Jewelry Accessories,Jewelry Boxes,None,None
2,2,1608299953,None,unavailable,Access for up to 5 family members Download act...,,Learn French: Rosetta Stone French - Level 1,missing,Software,Education & Reference,Languages,None,None,None
3,3,1617160377,None,unavailable,Access for up to 5 family members Download act...,,Learn Italian: Rosetta Stone Italian - Level 1,missing,Software,Education & Reference,Languages,None,None,None
4,4,B00001W0KA,None,Buzz Lightyear,Lead Free Child (4-6 & 7-8) Includes: Bodysuit...,,Buzz Lightyear Boy's Deluxe Toy Story Costume,SelectM(3T-4T)3T-4TSmall (4-6)Medium (7-8)Medi...,Toys & Games,Dress Up & Pretend Play,Costumes,None,None,None


In [13]:
redo_df.isnull().sum()

NameError: name 'redo_df' is not defined

In [15]:
redo_df = total_product[total_product['department'] == 'unknown category']

In [16]:
redo_df.shape

(12009, 14)

In [17]:
redo_list = list(redo_df['asin'])
redo_list

['B00005TQI7',
 'B0000722HK',
 'B00007GD9I',
 'B0000864CZ',
 'B0000866JI',
 'B0000867AN',
 'B0000867GG',
 'B0000867ON',
 'B00008695M',
 'B0000869QI',
 'B000086Q9O',
 'B00008ID0B',
 'B00008ID1O',
 'B00008ID39',
 'B00008IOHR',
 'B00008KI3U',
 'B00008LUS5',
 'B00009QB5V',
 'B0000AT3NT',
 'B0000B35D8',
 'B0000TIIRS',
 'B0000X5EJO',
 'B0000X5FAM',
 'B0000YEC8C',
 'B0000YECC8',
 'B0000ZCH4W',
 'B000147UNU',
 'B00014861K',
 'B00018HRU2',
 'B0001IA06U',
 'B0001VJ2V6',
 'B0001VWSFS',
 'B0001WD5RC',
 'B0001XH7X4',
 'B0001YWBG6',
 'B000200JQI',
 'B00020ZUC6',
 'B00023K3YS',
 'B00029BKAI',
 'B0002DG7ZM',
 'B0002EJ4XS',
 'B0002FHFZQ',
 'B0002FHGBO',
 'B0002FHIM6',
 'B0002FHIMQ',
 'B0002FHIVM',
 'B0002FHIWQ',
 'B0002JA0WC',
 'B0002L01YC',
 'B0002LKLPQ',
 'B0002NZ0LO',
 'B0002TVB70',
 'B0002U71R8',
 'B0002VG2XG',
 'B0002XROVS',
 'B0002ZOQZ8',
 'B00031XFL2',
 'B00061RG1O',
 'B00063J8MW',
 'B00063W3GA',
 'B00064UU1Y',
 'B000662EL6',
 'B00066G516',
 'B000673JT6',
 'B00068VGOU',
 'B0006GK8V4',
 'B0006IXC

In [16]:
len(redo_list)

12009

In [26]:
# redo2000 = pd.DataFrame(list_of_dicts)


In [27]:
redo2000.isnull().sum()

asin              0
category          0
color             0
description       0
details        1402
name              0
size           1603
dtype: int64

In [28]:
EDA(redo2000)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B00005TQI7,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
1,B0000722HK,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
2,B00007GD9I,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
3,B0000864CZ,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
4,B0000866JI,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None


In [50]:
more_redos = (redo2000[redo2000['department'] == 'unknown category'])
redo_list2 = list(more_redos['asin'])
len(redo_list2)

1402

In [33]:
(redo2000[redo2000['department'] != 'unknown category']).to_csv('data/redo1.csv')

In [43]:
#redo6000 = pd.DataFrame(list_of_dicts)
redo6000.head()

,asin,category,color,description,details,name,size
0,B0018NHASI,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",100% cotton. Machine wash and dry. Imported.\n...,[],Jockey Men's Underwear Classic Boxer Brief - 6...,"<div class=""a-section a-spacing-base variation..."
1,B0018NHD9Y,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",This fits your .\n \n\n\n Make sure this fi...,[],Body Glove Men's Riptide Water Shoe,"<div class=""a-section a-spacing-small"" id=""var..."
2,B0018ODO9G,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",100% Cotton\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n \n...,[],Levi's Men's Silver Tab Baggy Jean,"<div class=""a-section a-spacing-base variation..."
3,B0018OFRK0,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",100% Cotton\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n \n...,[],Levi's Men's Big and Tall 501 Original Fit Jean,"<div class=""a-section a-spacing-base variation..."
4,B0018OFTA8,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",100% Leather\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n \...,[],Levi's Men's Leather Belt With Plaque Buckle,"<div class=""a-section a-spacing-base variation..."


In [45]:
redo6000.isnull().sum()

asin              0
category          0
color             0
description       0
details        2811
name              0
size           3212
dtype: int64

In [46]:
EDA(redo6000)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B0018NHASI,Underwear,missing,100% cotton. Machine wash and dry. Imported. U...,missing,Jockey Men's Underwear Classic Boxer Brief - 6...,missing,"Clothing, Shoes & Jewelry",Men,Clothing,Boxer Briefs,None,None
1,B0018NHD9Y,Shoes,missing,This fits your . Make sure this fitsby ent...,missing,Body Glove Men's Riptide Water Shoe,missing,"Clothing, Shoes & Jewelry",Men,"Surf, Skate & Street",Athletic,Water Shoes,None
2,B0018ODO9G,Jeans,missing,100% Cotton Imported Machine Wash Whether it's...,missing,Levi's Men's Silver Tab Baggy Jean,missing,"Clothing, Shoes & Jewelry",Men,Clothing,None,None,None
3,B0018OFRK0,Jeans,missing,100% Cotton Imported Button Fly closure Wash A...,missing,Levi's Men's Big and Tall 501 Original Fit Jean,missing,"Clothing, Shoes & Jewelry",Men,Clothing,None,None,None
4,B0018OFTA8,Novelty,missing,100% Leather PERFECT BELT FOR YOUR NEXT BARBEC...,missing,Levi's Men's Leather Belt With Plaque Buckle,missing,"Clothing, Shoes & Jewelry",Novelty & More,Clothing,Men,Accessories,Belt Buckles


In [52]:
redo_list2.extend(list((redo6000[redo6000['department'] == 'unknown category'])['asin']))
len(redo_list2)

4213

In [53]:
(redo6000[redo6000['department'] != 'unknown category']).to_csv('data/redo2.csv')

In [21]:
#redo6455 = pd.DataFrame(list_of_dicts)

In [22]:
EDA(redo6455)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B007JBJPOA,"Lingerie, Sleep & Lounge",missing,nylon shapewear-briefs closure Light control f...,missing,Barely There Second Skinnies Smoothers Boxer,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Lingerie,Shapewear,Thigh Slimmers
1,B007JEHD5A,Wigs,unavailable,Products: 1 long Straight wig (100cm/39') + Wi...,missing,Hunnt® Long Cosplay Party Red Straight Wig 100cm,missing,"Clothing, Shoes & Jewelry",Costumes & Accessories,Women,None,None,None
2,B007JEI2N2,Pumps,missing,100% Leather Synthetic sole Heel measures appr...,missing,Naturalizer Women's Joni Pump,missing,"Clothing, Shoes & Jewelry",Women,Shoes,None,None,None
3,B007JF0UEK,Pants,missing,"94% Cotton, 6% Spandex Imported Machine Wash S...",missing,Lee Women's Perfect Fit Nancy Barely Bootcut Pant,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Casual,None,None
4,B007JF0UFO,Pants,unavailable,"Black, Carbon Rinse, Indigo Rinse, Raven, Whit...",missing,Lee Women's Midrise Vivian Trouser,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Casual,None,None


In [23]:
(redo6455[redo6455['department'] != 'unknown category']).to_csv('data/redo3.csv')

In [21]:
# redo8528 = pd.DataFrame(list_of_dicts)

In [22]:
EDA(redo8528)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B007WA2TCS,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
1,B007WA2U7W,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
2,B007WA2VB2,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
3,B007WA2ZTA,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
4,B007WA3558,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None


In [28]:
redo8528.shape

(2073, 13)

In [34]:
(redo8528[redo8528['name'] != 'unknown product name']).shape

(950, 13)

In [23]:
(redo8528[redo8528['department'] != 'unknown category']).to_csv('data/redo4.csv')

In [35]:
(redo8528[redo8528['department'] != 'unknown category']).shape

(950, 13)

In [36]:
#redo10000 = pd.DataFrame(list_of_dicts)
redo10000.head()

,asin,category,color,description,details,name,size
0,B009L4OGDS,"Clothing, Shoes & Jewelry\n \n\n\n ...",unavailable,6mm Crystal Ball Stud Earrings\n\t\t\t\t\t\t\t...,"[<td class=""a-span7 a-size-base""> ...",Sterling Silver Pink Crystal Ball Stud Earrings,None
1,B009L5081Q,"Clothing, Shoes & Jewelry\n \n\n\n ...",unavailable,Size: One size only (small)\n\t\t\t\t\t\t\t\n\...,[],Ninimour- Fashion Trendy Women's Stretchy Legg...,None
2,B009L53QXI,"Clothing, Shoes & Jewelry\n \n\n\n ...","[<div class=""a-row a-spacing-micro""> <label cl...",Synthetic\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n \n\t...,[],Dark Tan Faux Leather Bold Red Back Zipper Buc...,"<div class=""a-section a-spacing-base variation..."
3,B009L5YU7O,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",Leathette\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n \n\t...,[],MG Collection Yelena Top Handle Soft Hobo Shou...,None
4,B009LB8CX6,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",Made in USA or Imported\n\t\t\t\t\t\t\t\n\t\t\...,[],Kirkland Signature Outdoor Trail Sock Merino W...,"<div class=""a-section a-spacing-small"" id=""var..."


In [38]:
EDA(redo10000)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B009L4OGDS,Earrings,unavailable,6mm Crystal Ball Stud Earrings Plain Post and ...,missing,Sterling Silver Pink Crystal Ball Stud Earrings,missing,"Clothing, Shoes & Jewelry",Women,Jewelry,Stud,None,None
1,B009L5081Q,Novelty,unavailable,Size: One size only (small) Total Length: Abou...,missing,Ninimour- Fashion Trendy Women's Stretchy Legg...,missing,"Clothing, Shoes & Jewelry",Novelty & More,Clothing,Women,Leggings,None
2,B009L53QXI,Boots,missing,Synthetic Made in USA or Imported Synthetic so...,missing,Dark Tan Faux Leather Bold Red Back Zipper Buc...,missing,"Clothing, Shoes & Jewelry",Women,Shoes,Mid-Calf,None,None
3,B009L5YU7O,Shoulder Bags,missing,Leathette Imported fabric lining Snap closure ...,missing,MG Collection Yelena Top Handle Soft Hobo Shou...,missing,"Clothing, Shoes & Jewelry",Women,Handbags & Wallets,None,None,None
4,B009LB8CX6,Active,missing,Made in USA or Imported Merino wool blend Fibe...,missing,Kirkland Signature Outdoor Trail Sock Merino W...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Athletic Socks,None,None


In [44]:
(redo10000[redo10000['name'] != 'unknown product name']).to_csv('data/redo5.csv')

In [43]:
(redo10000[redo10000['name'] == 'unknown product name'])

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
8,B009LEFPAQ,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
10,B009LEFR4A,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
11,B009LEFS26,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
12,B009LEG2EO,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
13,B009LEG3XO,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
16,B009LM7BWS,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
17,B009LNATJY,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
22,B009LZTK10,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
24,B009M8UF38,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
28,B009MAW0A2,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None


In [54]:
redo8528.to_csv('redo_list1.csv')

In [55]:
redo10000.to_csv('redo_list2.csv')

In [51]:
redo_list2 = (list((redo8528[redo8528['department'] == 'unknown category'])['asin'])) + (list((redo10000[redo10000['department'] == 'unknown category'])['asin']))

len(redo_list2)


2099

In [53]:
redo_list2

['B007WA2TCS',
 'B007WA2U7W',
 'B007WA2VB2',
 'B007WA2ZTA',
 'B007WA3558',
 'B007WA38IM',
 'B007WA38OG',
 'B007WA397M',
 'B007WA3B5C',
 'B007WA3CUQ',
 'B007WA3GMK',
 'B007WA3HLK',
 'B007WA3JE0',
 'B007WA3PD0',
 'B007WA48E0',
 'B007WA49DA',
 'B007WA4CQ4',
 'B007WACT6Y',
 'B007WACX3I',
 'B007WACY5K',
 'B007WAD2QU',
 'B007WADBUC',
 'B007WADJ7M',
 'B007WADN4G',
 'B007WADRBK',
 'B007WADUOY',
 'B007WAEB10',
 'B007WAEBPQ',
 'B007WAEU5C',
 'B007WAEV9C',
 'B007WAF02E',
 'B007WAF8TO',
 'B007WAF9LQ',
 'B007WAFEWU',
 'B007WAFEYI',
 'B007WAFIEE',
 'B007WAFIGM',
 'B007WAFN4E',
 'B007WAFZTW',
 'B007WASL66',
 'B007WAT3I6',
 'B007WAT69W',
 'B007WAT8IG',
 'B007WATABQ',
 'B007WATBLU',
 'B007WATI18',
 'B007WAU1VY',
 'B007WAU2OK',
 'B007WB4OIY',
 'B007WN8MEE',
 'B007WO1PJW',
 'B007WPH9BY',
 'B007WPHD92',
 'B007WPHJRS',
 'B007WPHRVG',
 'B007WPHUYU',
 'B007WQY5W4',
 'B007WZJYTY',
 'B007X43KX0',
 'B007X44HTQ',
 'B007X44WV4',
 'B007X49DUY',
 'B007X734FG',
 'B007X73VAO',
 'B007XBE0BE',
 'B007XD4NCI',
 'B007XD4T